# Find momentum resolution

In [4]:
# file_name_au =  "nts.mu2e.CeEndpointOnSpillTriggered.MDC2020au_best_v1_3_v06_03_00.001210_00000339.root" # 
# file_name_au = "nts.mu2e.CosmicCRYSignalAllOffSpillTriggered-LH.MDC2020as_best_v1_3_v06_03_00.001202_00050440.root" #
file_list_au =  "/exp/mu2e/data/users/sgrant/mu2e-cosmic/filelists/nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020au_best_v1_3_v06_03_00.local.10.txt"

In [5]:
from pyutils.pyprocess import Processor
processor = Processor(use_remote=False) # , location="disk")

[pyprocess] ⭐️ Initialised Processor:
	path = 'EventNtuple/ntuple'
	use_remote = False
	verbosity=1


In [6]:
branches={
    "trk" : ["trk.pdg"],
    "trkfit" : ["trksegs"],
    "trkmc": ["trkmcsim"]
}
data = processor.process_data(
    # file_name=file_name_au, 
    file_list_path=file_list_au,
    branches=branches
)

[pyprocess] ⭐️ Loading file list from /exp/mu2e/data/users/sgrant/mu2e-cosmic/filelists/nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020au_best_v1_3_v06_03_00.local.10.txt
[pyprocess] ✅ Successfully loaded file list
	Path: None
	Count: 10 files
[pyprocess] ⭐️ Starting processing on 10 files with 10 threads


Processing: 100%|██████████████████████████████| 10/10 [00:06<00:00,  1.47file/s, successful=10, failed=0]


[pyprocess] ✅ Returning concatenated array containing 26959 events


In [8]:
from pyutils.pyprint import Print
printer = Print()
Print(verbose=False).print_n_events(data, 1)

[pyprint] ⭐️ Initialised Print with verbose = False and precision = 1
[pyprint] ⭐️ Initialised Print with verbose = False and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trk.trk.pdg: [-11, 11, -11, 11, -13, 13, -13, 13]
trkfit.trksegs.mom.fCoordinates.fX: [[-33.8, 59.3, 32.7, 57.5, -26.2, -60.2], ..., [55, 25.9, ..., -34.5, -42.9]]
trkfit.trksegs.mom.fCoordinates.fY: [[-71.6, 43, -66.1, 37.8, 64, 32.1], ..., [-43.3, -66.6, -62.1, ..., 58, 51.1]]
trkfit.trksegs.mom.fCoordinates.fZ: [[-18.7, 35.2, 34.1, 42.9, 40.9, 41.6], ..., [-40.3, -41.1, ..., 39.9, 40.5]]
trkfit.trksegs.pos.fCoordinates.fX: [[94.9, -215, 88.5, -207, -334, -234], ..., [-268, -339, -301, ..., 88.5, 80.4]]
trkfit.trksegs.pos.fCoordinates.fY: [[-434, -209, -287, -217, -506, -628], ..., [-601, -497, -317, ..., -287, -269]]
trkfit.trksegs.pos.fCoordinates.fZ: [[-3.9e+03, -3.09e+03, -2.92e+03, -2.42e+03, 10.1, 1.64e+03], [...], ...,

Sometimes you get situations like this: 

* some tracks have reflections, so you get same surfaces hit multiple times within a track segment 
* some tracks have electron showers which all produce the maximum number of hits. 

I would like to avoid putting a requirement on "no reflections" and things like that, since that falls under "preselection". 

I think we need to take the first trkmcsim and then broadcast that so that it matches the shape of trksegs. 

```
-------------------------------------------------------------------------------------
trk.pdg: [11, 11, 11, 11, 11, 11, 11, 11]
-------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------
trkmcsim.valid: [[True], [True, True, True], [True], [True, ...], ..., [True], [True], [True]]
trkmcsim.id: [[81], [40, 24, 246], [81], [101, 81], [25], [81], [81], [33]]
trkmcsim.nhits: [[23], [24, 24, 24], [18], [12, 12], [35], [23], [17], [12]]
trkmcsim.nactive: [[3], [7, 4, 1], [4], [3, 1], [6], [5], [3], [6]]
trkmcsim.rank: [[1], [0, 1, 4], [0], [0, 3], [0], [0], [1], [0]]
trkmcsim.pdg: [[11], [11, 11, 11], [11], [11, 11], [11], [11], [11], [11]]
trkmcsim.startCode: [[13], [17, 31, 13], [13], [13, 13], [31], [13], [13], [17]]
trkmcsim.stopCode: [[17], [17, 17, 17], [17], [17, 17], [17], [17], [17], [17]]
trkmcsim.gen: [[-1], [-1, -1, -1], [-1], [-1, -1], [-1], [-1], [-1], [-1]]
trkmcsim.time: [[5.33e+04], [5.33e+04, 5.33e+04, 5.33e+04], ..., [5.33e+04], [5.34e+04]]
trkmcsim.index: [[4], [0, 3, 6], [0], [0, 7], [0], [0], [4], [0]]
trkmcsim.mom.fCoordinates.fX: [[-69.6], [-0.182, -45.9, -73.3], [-69.6], ..., [-69.6], [-69.6], [-2.63]]
trkmcsim.mom.fCoordinates.fY: [[-71.2], [1.3, -48.7, -74.2], [-71.2], [...], ..., [-71.2], [-71.2], [-0.0454]]
trkmcsim.mom.fCoordinates.fZ: [[-32.3], [-0.709, -14.3, -38.3], [-32.3], [...], ..., [-32.3], [-32.3], [1.36]]
trkmcsim.pos.fCoordinates.fX: [[285], [516, 265, 281], [285], [280, 285], [326], [285], [285], [454]]
trkmcsim.pos.fCoordinates.fY: [[316], [122, 297, 313], [316], [311, 316], [358], [316], [316], [376]]
trkmcsim.pos.fCoordinates.fZ: [[1.72e+03], [772, 1.71e+03, 1.72e+03], [1.72e+03], ..., [1.72e+03], [-592]]
trkmcsim.endmom.fCoordinates.fX: [[0], [-0, -0, -0], [0], [0, 0], [-0], [0], [0], [0]]
trkmcsim.endmom.fCoordinates.fY: [[-0], [0, -0, -0], [-0], [-0, -0], [0], [-0], [-0], [-0]]
trkmcsim.endmom.fCoordinates.fZ: [[0], [0, -0, 0], [0], [0, 0], [0], [0], [0], [-0]]
trkmcsim.endpos.fCoordinates.fX: [[359], [513, 467, 384], [359], [399, 359], [449], [359], [359], [472]]
trkmcsim.endpos.fCoordinates.fY: [[172], [124, 111, 102], [172], [-37.8, 172], [350], [172], [172], [367]]
trkmcsim.endpos.fCoordinates.fZ: [[1.76e+03], [-398, 1.71e+03, 1.74e+03], ..., [1.76e+03], [1.11e+03]]
trkmcsim.prirel._rel: [[-1], [-1, -1, -1], [-1], [-1, -1], [-1], [-1], [-1], [-1]]
trkmcsim.prirel._rem: [[-1], [-1, -1, -1], [-1], [-1, -1], [-1], [-1], [-1], [-1]]
trkmcsim.trkrel._rel: [[0], [0, 0, 0], [0], [0, 0], [0], [0], [0], [0]]
trkmcsim.trkrel._rem: [[0], [0, 0, 0], [0], [0, 0], [0], [0], [0], [0]]
-------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------
trksegs.mom.fCoordinates.fX: [[59.2], [61.3], [30.1], [-7.99], ..., [-33], [47.3, -54.8], [33.2, 9.34]]
trksegs.mom.fCoordinates.fY: [[-29.9], [16.8], [48.8], [75.7], ..., [-30.5], [31.3, 11.9], [-56.3, 62.2]]
trksegs.mom.fCoordinates.fZ: [[224], [399], [282], [419], [-277], [-147], [-41.8, 41.3], [-44.7, 47.7]]
trksegs.pos.fCoordinates.fX: [[130], [368], [459], [672], [254], [307], [533, 463], [110, 477]]
trksegs.pos.fCoordinates.fY: [[47.7], [-37.6], [139], [151], [273], [399], [82.9, 421], [165, 251]]
trksegs.pos.fCoordinates.fZ: [[-1.63e+03], [-1.63e+03], ..., [-1.63e+03, ...], [-1.63e+03, -1.63e+03]]
trksegs.time: [[5.33e+04], [5.33e+04], ..., [5.33e+04, 5.33e+04], [5.34e+04, 5.34e+04]]
trksegs.dmom: [[0], [0], [0], [0], [0], [0], [0, 0], [0, 0]]
trksegs.momerr: [[1.54], [6.39], [3.26], [4.29], ..., [0.744], [0.13, 0.138], [0.135, 0.185]]
trksegs.inbounds: [[True], [True], [True], [True], ..., [True], [True, True], [True, True]]
trksegs.gap: [[False], [False], [False], ..., [False], [False, False], [False, False]]
trksegs.early: [[False], [False], [False], ..., [False], [False, False], [False, False]]
trksegs.late: [[False], [False], [False], [False], ..., [False], [False, True], [False, True]]
trksegs.sid: [[0], [0], [0], [0], [0], [0], [0, 0], [0, 0]]
trksegs.sindex: [[0], [0], [0], [0], [0], [0], [0, 0], [0, 0]]
-------------------------------------------------------------------------------------
```

In [205]:
# trkmc = ak.Array([[1], [0, 1, 4], [0], [0, 3], [0], [0], [1], [0]])
# trkfit = ak.Array([[0], [0], [0], [0], [0], [0], [0, 0], [0, 0]])

# a = [2] - [None]

In [9]:
from pyutils.pyselect import Select
from pyutils.pyvector import Vector
from pyutils.pylogger import Logger
from pyutils.pyprint import Print
import awkward as ak
class Test:
    def __init__(self):
        
        # self.analyse = analyse
        # Get attributes from analyse instance
        # self.thresholds = analyse.thresholds
        self.selector = Select() # selector
        self.vector = Vector() # analyse.vector
        self.verbosity = 2
        self.printer = Print()

        # Seperate logger
        self.logger = Logger(
            print_prefix="[Test]",
            verbosity=self.verbosity
        )

    
    def _prepare_track_data(self, data, pad=False):
        """
        Loose cuts

        We have to select electron fits at the tracker entrance AND THAT'S IT!
        
        Returns:
            tuple: (trk, trkfit, trkmcsim)

        """

        # Select electron tracks
        is_reco_electron = self.selector.is_electron(data["trk"])

        # Select tracker front
        at_trk_front = self.selector.select_surface(data["trkfit"], surface_name="TT_Front")
        has_trk_front = ak.any(at_trk_front, axis=-1)

        # MC truth selections
        # Truth track is electron 
        is_truth_electron = data["trkmc"]["trkmcsim"]["pdg"] == 11
        # is_trk_parent = data["trkmc"]["trkmcsim"]["nhits"] == ak.max(data["trkmc"]["trkmcsim"]["nhits"], axis=-1)
        # is_trk_parent_electron = is_truth_electron & is_trk_parent 
        # has_trk_parent_electron = ak.any(is_trk_parent_electron, axis=-1)

        # Copy data
        data_cut = ak.copy(data)
    
        # Construct combined trk level mask 
        trk_mask = is_reco_electron & has_trk_front # & has_trk_parent_electron
        
        # Event level mask 
        has_trks = ak.any(trk_mask, axis=-1)

        # # Apply trksegs-level selection
        data_cut["trkfit"] = data_cut["trkfit"][at_trk_front]

        # # # Apply trkmcsim-level selection
        data_cut["trkmc"] = data_cut["trkmc"][is_truth_electron]

        # Apply trk-level selection 
        data_cut["trk"]  = data_cut["trk"][trk_mask]
        data_cut["trkfit"] = data_cut["trkfit"][trk_mask]
        data_cut["trkmc"] = data_cut["trkmc"][trk_mask]

        # Apply event-level selection
        data_cut = data_cut[has_trks]

        # Complex EM showers and interesting reflections screw everything up 
        # trksegs and trkmcsim get misaligned and there is no easy way to fix it

        
        # In this case we can just pad the trkmc and trkseg branches with None 
        # 

        # Clean up 

        # # Not sure if needed
        # has_trks = ak.any(trk["trk.pdg"], axis=-1)
        # print(has_trks)
        # trk = trk[has_trks]
        # trkfit = trkfit[has_trks]
        # trkmc = trkmc[has_trks]

        # # Certain files
        # has_trk_segs = ak.num(trkfit["trksegs"], axis=-1) > 0
        # print(has_trk_segs)
        # print(trk["trk.pdg"])
        # # has_trk_segs = ak.any(has_trk_segs, axis=-1)
        # print(has_trk_segs)
        # trk = trk[has_trk_segs]
        # trkfit = trkfit[has_trk_segs]
        # trkmc = trkmc[has_trk_segs]      

        # Get the mask at the right level
        # has_trk_segs = ak.num(trkfit["trksegs"], axis=-1) > 0  # [event : track]
        
        # # Apply to each array at the correct axis
        # trk = trk[has_trk_segs]           # [event : track] -> [event : track]  
        # trkfit = trkfit[has_trk_segs]     # [event : track : intersection] -> [event : track : intersection]
        # trkmc = trkmc[has_trk_segs]       # [event : track : genealogy] -> [event : track : genealogy]

        # MC truth selections
        # Truth track parent is electron 
        # We must do this after the reco cuts, or 
        # # the resulting trkmc array will not align with trksegs
        # is_truth_electron = trkmc["trkmcsim"]["pdg"] == 11
        # is_trk_parent = trkmc["trkmcsim"]["nhits"] == ak.max(trkmc["trkmcsim"]["nhits"], axis=-1)
        # is_trk_parent_electron = is_truth_electron & is_trk_parent 
    
        # # Apply MC genealogy-level selections
        # trkmc = trkmc[is_trk_parent_electron]

        # Clean up
        # # Not sure if needed
        # has_trk_parent_electron = ak.any(is_trk_parent_electron, axis=-1)
        # trk = trk[has_trk_parent_electron]
        # trkfit = trkfit[has_trk_parent_electron]
        # trkmc = trkmc[has_trk_parent_electron]
        
        return data_cut["trk"], data_cut["trkfit"], data_cut["trkmc"] #[:,:,0]

    def _extract_data(self, data, param):
        if param == "mom_res":
            trk, trkfit, trkmc = self._prepare_track_data(data)

            # print("*"*50)
            # print("trkfit")
            # printer.print_n_events(trk, 5)
            
            # print("*"*50)
            # print("trkfit")
            # printer.print_n_events(trkfit, 5)
    
            # print("*"*50)
            # print("trkmc")
            # printer.print_n_events(trkmc, 5)
        
            mom_reco = self.vector.get_mag(trkfit["trksegs"], "mom")
            mom_truth = self.vector.get_mag(trkmc["trkmcsim"], "mom")
            mom_reco = ak.flatten(mom_reco,axis=None) 
            mom_truth = ak.flatten(mom_truth,axis=None)
            try:
                mom_res = mom_reco - mom_truth
                return mom_res
            except:
                self.logger.log(f"Misalignment in momentum resolution calculation:\n\tmom_reco has length {len(mom_reco)}\n\tmom_truth has length {len(mom_truth)}", "warning")
                return []
            
        

test = Test()

trk, trkfit, trkmc = test._prepare_track_data(data) # , "mom_res")

# has_trk_segs = ak.num(trkfit["trksegs"], axis=-1)  > 0
# has_trk_mc = ak.num(trkmc["trkmcsim"], axis=-1) > 0

# n_trk_segs = ak.count(trkfit["trksegs"]["sid"], axis=None)
# n_trk_mc = ak.count(trkmc["trkmcsim"]["index"], axis=None)

# print(n_trk_segs)
# print(n_trk_mc)

results = test._extract_data(data, "mom_res")
# print(len(trk), len(trkfit), len(trkmc))


[pyvector] ⭐️ Initialised Vector with verbosity = 1
[pyprint] ⭐️ Initialised Print with verbose = False and precision = 1
[pyselect] ✅ Returning mask for e- tracks
[pyselect] ✅ Returning mask for trksegs with sid = 0
[pyselect] ✅ Returning mask for e- tracks
[pyselect] ✅ Returning mask for trksegs with sid = 0
[pyvector] ✅ Created 3D 'mom' vector
[pyvector] ✅ Got 'mom' magnitude
[pyvector] ✅ Created 3D 'mom' vector
[pyvector] ✅ Got 'mom' magnitude
[Test] ⚠️ Misalignment in momentum resolution calculation:
	mom_reco has length 37579
	mom_truth has length 69054


In [10]:
Print().print_n_events(trk)
Print().print_n_events(trkfit)
Print().print_n_events(trkmc)

[pyprint] ⭐️ Initialised Print with verbose = False and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trk.pdg: [11, 11]
-------------------------------------------------------------------------------------

[pyprint] ⭐️ Initialised Print with verbose = False and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trksegs.mom.fCoordinates.fX: [[63.5], [-31.8, 60]]
trksegs.mom.fCoordinates.fY: [[24.5], [-61.3, 32.7]]
trksegs.mom.fCoordinates.fZ: [[43.7], [-43, 43.5]]
trksegs.pos.fCoordinates.fX: [[-3.91], [-300, 21.4]]
trksegs.pos.fCoordinates.fY: [[-606], [-303, -601]]
trksegs.pos.fCoordinates.fZ: [[-1.63e+03], [-1.63e+03, -1.63e+03]]
trksegs.time: [[961], [909, 961]]
trksegs.dmom: [[0], [0, 0]]
trksegs.momerr: [[0.0428], [0.107, 0.167]]
trksegs.inbounds: [[True], [True, True]]
trksegs.gap: [[False], [False, False

In [210]:
# has_trk_segs = ak.num(trkfit["trksegs"], axis=-1)  > 0
# has_trk_mc = ak.num(trkmc["trkmcsim"], axis=-1) > 0

# n_trk_segs = ak.count(has_trk_segs, axis=None)
# n_trk_mc = ak.count(has_trk_mc, axis=None)
# print(n_trk_segs)
# print(n_trk_mc)

In [ ]:
print(len(trk))
print(len(trkfit))
print(len(trkmc))

In [ ]:
print(len(trkfit[ak.any(trkfit["trksegs"]["sid"], axis=-1)]))
print(len(trkmc[ak.any(trkmc["trkmcsim"]["time"], axis=-1)]))

In [ ]:
print(ak.type(trk))
print(ak.type(trkfit))
print(ak.type(trkmc))

In [ ]:
# print(trkfit[ak.num(trkfit["trksegs"]["sid"], axis=-1)])
# print(trkmc[ak.num(trkmc["trkmcsim"]["time"], axis=-1)])

In [ ]:
import matplotlib.pyplot as plt 
from pyutils.pyplot import Plot
plotter = Plot() 
fig, ax = plt.subplots(1, 2, figsize=(2*6.4, 4.8))
plotter.plot_1D(results, nbins=140, xmin=-5, xmax=2, col="blue", log_y=False, xlabel=r"$\delta p = p_{\text{reco}} - p_{\text{truth}}$ [MeV/c]", ylabel = "Tracks / 50 keV/c", ax=ax[0], show=False)
plotter.plot_1D(results, nbins=140, xmin=-5, xmax=2, col="blue", log_y=True, xlabel=r"$\delta p = p_{\text{reco}} - p_{\text{truth}}$ [MeV/c]", ax=ax[1], show=False)
plt.tight_layout()
out_path="../../output/images/dev/h1_1x2_mom_res_au_single.png"
plt.savefig(out_path, dpi=300)
plt.show()
print(f"Wrote {out_path}")

In [ ]:
# !mkdir ../../output/images/dev

In [101]:
# !mkdir ../../output/images/dev/